# Análisis de Patrones de Correos
Este notebook descarga correos históricos para identificar casos borde en el parser.

In [1]:
import pandas as pd
from src.ingestion import GmailClient
from src.parser import TransactionParser
import logging

# Configurar pandas para ver todo el texto
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

c:\Users\Administrador\Documents\Proyectos\autotrx\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Administrador\Documents\Proyectos\autotrx\src\parser.py:6: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [2]:
# 1. Inicializar Cliente
client = GmailClient()
parser = TransactionParser()

print("Autenticación exitosa.")

Autenticación exitosa.


In [ ]:
# 2. Descargar Historial (últimos 50 correos de Bancolombia)
# Ajusta el 'sender' o el 'custom_query' según necesites
# Ejemplo: "from:alertasynotificaciones@bancolombia.com.co" o "subject:Transferencia"

SEARCH_QUERY = "Bancolombia"  # Busca "Bancolombia" en general
MAX_EMAILS = 50

print(f"Buscando últimos {MAX_EMAILS} correos con query: '{SEARCH_QUERY}'...")

emails = client.fetch_unread_emails(
    max_results=MAX_EMAILS, 
    custom_query=SEARCH_QUERY
)

print(f"Descargados: {len(emails)}")

In [ ]:
# 3. Probar el Parser
results = []

for email in emails:
    # Preferimos el cuerpo, fallback al snippet
    text = email.get('body') or email.get('snippet', '')
    
    parsed = parser.parse(text)
    
    results.append({
        "id": email['id'],
        "date_extracted": parsed['date'],
        "amount": parsed['amount'],
        "merchant": parsed['merchant'],
        "original_snippet": email.get('snippet', '')[:100] + "...",
        "full_text": text[:200]  # Guardamos primeros 200 chars para análisis
    })

df = pd.DataFrame(results)

In [ ]:
# 4. Análisis de Casos Fallidos (Monto 0 o Merchant UNKNOWN)
failed_cases = df[ (df['amount'] == 0) | (df['merchant'] == 'UNKNOWN') ]

print(f"Total procesados: {len(df)}")
print(f"Casos exitosos: {len(df) - len(failed_cases)}")
print(f"Casos fallidos/dudosos: {len(failed_cases)}")

# Mostrar los fallidos para inspeccionar
print("\n--- CASOS QUE REQUIEREN ATENCIÓN ---")
display(failed_cases[['amount', 'merchant', 'full_text']])

In [ ]:
# 5. Exportar para revisión manual si deseas
# df.to_csv("analisis_correos.csv", index=False)

In [3]:
# 2. Descargar Historial (últimos 50 correos de RAPPICARD)
# Cambiamos el filtro al remitente de RappiCard

SEARCH_QUERY = "from:noreply@rappicard.co" # Remitente específico de Rappi
MAX_EMAILS = 50

print(f"Buscando últimos {MAX_EMAILS} correos de RAPPICARD...")

emails = client.fetch_unread_emails(
    max_results=MAX_EMAILS, 
    custom_query=SEARCH_QUERY
)

print(f"Descargados: {len(emails)}")

# Limpiamos el dataframe anterior para no mezclar datos si vuelves a correr celdas de abajo
df = pd.DataFrame() 

Buscando últimos 50 correos de RAPPICARD...
Fetching latest 50 emails with query: from:noreply@rappicard.co
Fetched 50 emails.
Descargados: 50


In [4]:
# 3. Probar el Parser (Mejorado con Metadatos)
results = []

def get_header(payload, name):
    """Extrae un header específico del payload del correo"""
    if not payload or 'headers' not in payload:
        return None
    for h in payload['headers']:
        if h['name'] == name:
            return h['value']
    return None

for email in emails:
    # Preferimos el cuerpo, fallback al snippet
    text = email.get('body') or email.get('snippet', '')
    
    # Extraemos metadatos reales del correo (independiente del parser)
    payload = email.get('payload', {})
    subject = get_header(payload, 'Subject')
    email_date = get_header(payload, 'Date')
    
    # Intentamos parsear
    parsed = parser.parse(text)
    
    results.append({
        "id": email['id'],
        "email_date": email_date,        # Fecha real del correo
        "subject": subject,              # Asunto (muy útil para distinguir tipos de correo)
        "amount": parsed['amount'],      # Lo que entendió el parser
        "merchant": parsed['merchant'],  # Lo que entendió el parser
        "full_text": text                # Texto COMPLETO para debuggear
    })

df = pd.DataFrame(results)

# Mostrar las primeras filas con las columnas más importantes primero
print(f"Procesados {len(df)} correos.")
column_order = ['email_date', 'subject', 'amount', 'merchant', 'full_text']
# Aseguramos que existan las columnas antes de ordenar (por si df está vacío)
if not df.empty:
    display(df[column_order].head())
else:
    print("No se encontraron correos.")

Regex failed to fully parse. Attempting fallback to Gemini...
LLM Success: {'date': '28/11/2025 11:12', 'amount': 2281289.86, 'merchant': 'RAPPICARD', 'description': 'RAPPICARD'}
Regex failed to fully parse. Attempting fallback to Gemini...
Error inside _parse_with_llm: float() argument must be a string or a real number, not 'NoneType'
Regex failed to fully parse. Attempting fallback to Gemini...
Error inside _parse_with_llm: float() argument must be a string or a real number, not 'NoneType'
Regex failed to fully parse. Attempting fallback to Gemini...
LLM Success: {'date': '30/09/2025 08:32', 'amount': 4378807.81, 'merchant': 'PAGO DE TARJETA DE CRÉDITO', 'description': 'PAGO DE TARJETA DE CRÉDITO'}
Regex failed to fully parse. Attempting fallback to Gemini...
Error inside _parse_with_llm: float() argument must be a string or a real number, not 'NoneType'
Procesados 50 correos.


email_date                             subject  \
0  Wed, 17 Dec 2025 20:22:25 +0000  RappiCard - Resumen de transacción   
1  Tue, 16 Dec 2025 02:51:02 +0000  RappiCard - Resumen de transacción   
2  Tue, 16 Dec 2025 02:17:17 +0000  RappiCard - Resumen de transacción   
3  Mon, 15 Dec 2025 11:45:03 +0000  RappiCard - Resumen de transacción   
4  Fri, 12 Dec 2025 06:34:05 +0000  RappiCard - Resumen de transacción   

      amount               merchant  \
0    3913.04     ORACLE AMERICA INC   
1   31267.00      CABIFYCO YRAH FAN   
2   79000.00  DLO GOOGLE GOOGLE ONE   
3   30900.00      EXPERIAN COLOMBIA   
4  142900.00                  APPLE   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [5]:
# 4. Análisis de Casos Fallidos (Monto 0 o Merchant UNKNOWN)
failed_cases = df[ (df['amount'] == 0) | (df['merchant'] == 'UNKNOWN') ]

print(f"Total procesados: {len(df)}")
print(f"Casos exitosos: {len(df) - len(failed_cases)}")
print(f"Casos fallidos/dudosos: {len(failed_cases)}")

# Mostrar los fallidos para inspeccionar
print("\n--- CASOS QUE REQUIEREN ATENCIÓN ---")
display(failed_cases[['amount', 'merchant', 'full_text']])

Total procesados: 50
Casos exitosos: 47
Casos fallidos/dudosos: 3

--- CASOS QUE REQUIEREN ATENCIÓN ---


,amount,merchant,full_text
13,0.0,UNKNOWN,"Ya puedes revisarlo y pagar tu tarjeta ¡Hola, JUAN MANUEL! Consulta y descarga tu extracto desde el app, seleccionando el año y el mes que deseas ver. Desbloquéalo con tu número de documento. Pago"
24,0.0,UNKNOWN,"Ya puedes revisarlo y pagar tu tarjeta ¡Hola, JUAN MANUEL! Consulta y descarga tu extracto desde el app, seleccionando el año y el mes que deseas ver. Desbloquéalo con tu número de documento. Pago"
41,0.0,UNKNOWN,"Ya puedes revisarlo y pagar tu tarjeta ¡Hola, JUAN MANUEL! Consulta y descarga tu extracto desde el app, seleccionando el año y el mes que deseas ver. Desbloquéalo con tu número de documento. Pago"
